In [15]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error

## Q1. Downloading the data

In [3]:
jan = pd.read_parquet('../data/yellow_tripdata_2022-01.parquet')
print(len(jan.columns))

19


## Q2. Computing duration


In [10]:
jan['duration'] = jan['tpep_dropoff_datetime'] - jan['tpep_pickup_datetime']
jan['duration'] = jan['duration'].dt.total_seconds() / 60
print(jan['duration'].std())

46.44530513776802


## Q3. Dropping outliers


In [11]:
jan_no_outliers = jan[(jan['duration'] >= 1) & (jan['duration'] <= 60)]
print(len(jan_no_outliers) / len(jan))

0.9827547930522406


## Q4. One-hot encoding


In [14]:
categorical_features = ['PULocationID', 'DOLocationID']

jan_no_outliers[categorical_features] = jan_no_outliers[categorical_features].astype(str)
train_dicts = jan_no_outliers[categorical_features].to_dict('records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_train

/var/folders/xm/8mvqw44j1md_q70lrkm9_wh00000gn/T/ipykernel_48680/865539698.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan_no_outliers[categorical_features] = jan_no_outliers[categorical_features].astype(str)


<2421440x515 sparse matrix of type '<class 'numpy.float64'>'
	with 4842880 stored elements in Compressed Sparse Row format>

In [16]:
from sklearn.linear_model import LinearRegression


y_train = jan_no_outliers['duration'].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

rmse = mean_squared_error(y_train, y_pred, squared=False)

print(rmse)


6.986190814952337


## Q6. Evaluating the model

In [21]:
def read_df(file_path):
    df = pd.read_parquet(file_path)
    
    df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['duration'] = df['duration'].dt.total_seconds() / 60
    
    df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]
    
    categorical_features = ['PULocationID', 'DOLocationID']
    df[categorical_features] = df[categorical_features].astype(str)
    return df

In [28]:
df_train = read_df('../data/yellow_tripdata_2022-01.parquet')
df_val = read_df('../data/yellow_tripdata_2022-02.parquet')

In [29]:
len(df_train), len(df_val)

(2421440, 2918187)

In [30]:
categorical = ['PULocationID', 'DOLocationID']

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict('records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict('records')
X_val = dv.transform(val_dicts)

In [31]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [32]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

rmse = mean_squared_error(y_val, y_pred, squared=False)
print(rmse)

7.786407163179794
